## 1. FIRST REQUEST TO PULL THE LIST OF YOUTUBE VIDEOS

### 1.1 IMPORT MODULES

In [88]:
import os
import requests as r
import json
from dotenv import load_dotenv
import pandas as pd
import pymysql as sql
from sqlalchemy import create_engine

In [64]:
load_dotenv()

True

### 1.2 API KEYS, QUERY PARAMETERS & THE ENDPOINT URL

In [65]:
api_key = os.getenv('api_key')
channel_id = os.getenv('channel_id')

In [66]:
base_url = 'https://www.googleapis.com/youtube/v3'

endpoint_1 = '/search'
full_url_1 = base_url + endpoint_1

In [67]:
# parameters for the first request
page_token = ''
part_1 = 'snippet,id'
order = 'date'
maxResults = '50'
q = 'messi'
type = 'video'
videoDefinition = 'high'

params_1 = {'key' : api_key, 
          'channelid' : channel_id, 
          'part' : part_1, 
          'order' : order, 
          'maxResults' : maxResults, 
          'page_token' : page_token,
           'q' : q,
           'type' : type,
           'videoDefinition' : videoDefinition}

### 1.3 SEND THE REQUEST AND FLATTEN THE JSON RESPONSE

In [68]:
response_1 = r.get(full_url_1, params = params_1).json()
data_1 = pd.json_normalize(response_1, record_path = ['items'], meta = ['kind', 'etag', 'nextPageToken', 'regionCode'], meta_prefix = 'meta')
data_1.head(1)

,kind,etag,id.kind,id.videoId,snippet.publishedAt,snippet.channelId,snippet.title,snippet.description,snippet.thumbnails.default.url,snippet.thumbnails.default.width,...,snippet.thumbnails.high.url,snippet.thumbnails.high.width,snippet.thumbnails.high.height,snippet.channelTitle,snippet.liveBroadcastContent,snippet.publishTime,metakind,metaetag,metanextPageToken,metaregionCode
0,youtube#searchResult,AAzyJ6lkkKvYTy8qhCqHEY5kUBM,youtube#video,-lDpm6VrGxA,2022-06-24T11:30:00Z,UCrY1RI5kdTwwpKAk5qozjJQ,Saving Messi From PSG😎 Messi PSG യിൽ പോയിരുന്ന...,amd #messi #fifa22 Haaloo Noobs Channel Supp...,https://i.ytimg.com/vi/-lDpm6VrGxA/default.jpg,120,...,https://i.ytimg.com/vi/-lDpm6VrGxA/hqdefault.jpg,480,360,AMD isPlaying,none,2022-06-24T11:30:00Z,youtube#searchListResponse,yk5e1wm3s7_hbqMkVBiCQyiIP9E,CDIQAA,AZ


### 1.4 DELETE AND RENAME THE COLUMNS

In [69]:
data_1 = data_1[['id.kind', 'id.videoId', 'snippet.publishedAt', 'snippet.channelId', 'snippet.title', 'snippet.description', 'snippet.channelTitle', 'metanextPageToken', 'metaregionCode']]

In [70]:
data_1 = data_1.rename(
        columns = {
            'id.kind':'kind', 
            'id.videoId':'video_id', 
            'snippet.publishedAt':'published_at', 
            'snippet.channelId':'channel_id',
            'snippet.title':'title',
            'snippet.description':'description',
            'snippet.channelTitle':'channel_title',
            'metanextPageToken':'next_page_token',
            'metaregionCode':'region_code'
        }
    )

## 2. THE SECOND REQUEST TO EXTEND THE FIRST REQUEST WITH METRICS

### 2.1 QUERY PARAMETERS & THE ENDPOINT URL

In [71]:
endpoint_2 = '/videos'
full_url_2 = base_url + endpoint_2

In [72]:
# parameters for the second request
part_2 = 'statistics'
video_id = data_1['video_id']

params_2 = {'key' : api_key, 
            'id' : video_id, 
            'part' : part_2,
            'maxResults' : maxResults,
            'page_token' : page_token}

### 2.2 SEND THE REQUEST AND FLATTEN THE JSON RESPONSE

In [73]:
response_2 = r.get(full_url_2, params = params_2).json()
data_2 = pd.json_normalize(response_2, record_path = ['items'])
data_2.head(1)

,kind,etag,id,statistics.viewCount,statistics.likeCount,statistics.favoriteCount,statistics.commentCount
0,youtube#video,nqOVcdoCKjw-f3Pjghmz4bkyYXU,-lDpm6VrGxA,4527,1755,0,62


### 2.3 DELETE AND RENAME THE COLUMNS

In [74]:
data_2 = data_2[['id', 'statistics.viewCount', 'statistics.likeCount', 'statistics.favoriteCount', 'statistics.commentCount']]

In [75]:
data_2 = data_2.rename(
        columns = {
            'id':'video_id',
            'statistics.viewCount':'view_count', 
            'statistics.likeCount':'like_count', 
            'statistics.favoriteCount':'favorite_count', 
            'statistics.commentCount':'comment_count'
        }
    )

## 3. JOIN THE 2 DATASETS USING THE VIDEO_ID COLUMN

In [76]:
data_final = pd.merge(data_1, data_2, how = 'inner', on = 'video_id')
data_final.head(1)

,kind,video_id,published_at,channel_id,title,description,channel_title,next_page_token,region_code,view_count,like_count,favorite_count,comment_count
0,youtube#video,-lDpm6VrGxA,2022-06-24T11:30:00Z,UCrY1RI5kdTwwpKAk5qozjJQ,Saving Messi From PSG😎 Messi PSG യിൽ പോയിരുന്ന...,amd #messi #fifa22 Haaloo Noobs Channel Supp...,AMD isPlaying,CDIQAA,AZ,4527,1755,0,62


## 4. BASIC STATISTICS, DATA CLEANING AND DATA TYPE CHANGE

In [77]:
data_final.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 50 entries, 0 to 49
Data columns (total 13 columns):
 #   Column           Non-Null Count  Dtype 
---  ------           --------------  ----- 
 0   kind             50 non-null     object
 1   video_id         50 non-null     object
 2   published_at     50 non-null     object
 3   channel_id       50 non-null     object
 4   title            50 non-null     object
 5   description      50 non-null     object
 6   channel_title    50 non-null     object
 7   next_page_token  50 non-null     object
 8   region_code      50 non-null     object
 9   view_count       50 non-null     object
 10  like_count       50 non-null     object
 11  favorite_count   50 non-null     object
 12  comment_count    49 non-null     object
dtypes: object(13)
memory usage: 5.5+ KB


In [78]:
# delete unnecessary columns
data_final = data_final.dropna(subset = ['like_count'])
data_final = data_final.dropna(subset = ['comment_count'])

In [79]:
# improve data types
data_final['published_at'] = data_final['published_at'].astype('datetime64[ns]')
data_final['view_count'] = data_final['view_count'].astype('int')
data_final['favorite_count'] = data_final['favorite_count'].astype('int')
data_final['like_count'] = data_final['like_count'].astype('int')
data_final['comment_count'] = data_final['comment_count'].astype('int')

In [80]:
data_final.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 49 entries, 0 to 49
Data columns (total 13 columns):
 #   Column           Non-Null Count  Dtype         
---  ------           --------------  -----         
 0   kind             49 non-null     object        
 1   video_id         49 non-null     object        
 2   published_at     49 non-null     datetime64[ns]
 3   channel_id       49 non-null     object        
 4   title            49 non-null     object        
 5   description      49 non-null     object        
 6   channel_title    49 non-null     object        
 7   next_page_token  49 non-null     object        
 8   region_code      49 non-null     object        
 9   view_count       49 non-null     int32         
 10  like_count       49 non-null     int32         
 11  favorite_count   49 non-null     int32         
 12  comment_count    49 non-null     int32         
dtypes: datetime64[ns](1), int32(4), object(8)
memory usage: 4.6+ KB


In [81]:
data_final.describe()

,view_count,like_count,favorite_count,comment_count
count,49.000000,49.000000,49.0,49.000000
mean,35420.612245,2196.204082,0.0,140.000000
std,83797.097694,4406.169174,0.0,292.430334
min,365.000000,25.000000,0.0,0.000000
25%,4527.000000,224.000000,0.0,14.000000
50%,15428.000000,575.000000,0.0,41.000000
75%,34035.000000,2961.000000,0.0,102.000000
max,583601.000000,29181.000000,0.0,1858.000000


## 5. EXPORT DATA INTO A MYSQL DATABASE

In [82]:
host = os.getenv('host')
user = os.getenv('user')
password = os.getenv('password')
database = os.getenv('database')

In [83]:
conn_properties = {
    'host':host,
    'user':user,
    'password':password,
    'database':database,
    'local_infile':True,
    'charset':'utf8mb4'
}

In [84]:
mysql_conn = sql.connect(**conn_properties)
mysql_cursor = mysql_conn.cursor()

In [85]:
q1_createtable = 'create table if not exists search (kind varchar(60), video_id varchar(60), published_at datetime, channel_id varchar(60), title varchar(240), description varchar(240), channel_title varchar(60), next_page_token varchar(60), region_code varchar(3), view_count int, like_count int, favorite_count int, comment_count int, primary key(video_id))'
q1_exe = mysql_cursor.execute(q1_createtable)

In [86]:
# Create SQLAlchemy engine to connect to MySQL Database
engine = create_engine("mysql+pymysql://{user}:{password}@{host}/{database}".format(host = host, database = database, user = user, password = password))

In [87]:
# Convert dataframe to sql table                                   
data_final.to_sql('search', engine, index = False, if_exists = 'replace')

49